# Pricing estimate

## I. Bedrock

In [29]:
#Model: Amazon Nova Micro v1.0, prices in USD
price_1M_input = 0.04
price_1M_output = 0.14

#1 token ~4 characters

In [51]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

data_cnn = pd.read_json("analyzed_data/cnn_economic_sentiment.jsonl", orient="records", lines=True)
data_fox = pd.read_json("analyzed_data/fox_news_economic_sentiment.jsonl", orient="records", lines=True)

In [52]:
data_cnn["cnn_input"] = data_cnn["body"].str.len().fillna(0).astype(int)
data_cnn["cnn_input"] = data_cnn["cnn_input"].where(data_cnn["cnn_input"] <= 10000, 10000) #articles are truncated to 10 thousand characters in the code
data_fox["fox_input"] = data_fox["body"].str.len().fillna(0).astype(int)
data_fox["fox_input"] = data_fox["fox_input"].where(data_fox["fox_input"] <= 10000, 10000)

In [53]:
cnn_1st_input = np.sum(data_cnn["cnn_input"])
cnn_1st_input_cost = cnn_1st_input/1000000*price_1M_input
cnn_1st_output_cost = len(data_cnn)/1000000*price_1M_output #output is just ~1 token
print(f"Total input cost for determening wether the CNN {len(data_cnn):,} articles are related to economy or not: {cnn_1st_input_cost:.2f} USD")
print(f"Total output cost for determening wether the CNN {len(data_cnn):,} articles are related to economy or not: {cnn_1st_output_cost:.4f} USD")
print("="*120)
fox_1st_input = np.sum(data_fox["fox_input"])
fox_1st_input_cost = fox_1st_input/1000000*price_1M_input
fox_1st_output_cost = len(data_fox)/1000000*price_1M_output #output is just ~1 token
print(f"Total input cost for determening wether the FOX News {len(data_fox):,} articles are related to economy or not: {fox_1st_input_cost:.2f} USD")
print(f"Total output cost for determening wether the FOX News {len(data_fox):,} articles are related to economy or not: {fox_1st_output_cost:.4f} USD")

Total input cost for determening wether the CNN 2,809 articles are related to economy or not: 0.51 USD
Total output cost for determening wether the CNN 2,809 articles are related to economy or not: 0.0004 USD
Total input cost for determening wether the FOX News 1,160 articles are related to economy or not: 0.24 USD
Total output cost for determening wether the FOX News 1,160 articles are related to economy or not: 0.0002 USD


In [54]:
filtered_data_cnn = data_cnn.loc[data_cnn["is_economy"] == 1]
filtered_data_cnn["cnn_input"] = filtered_data_cnn["body"].str.len().fillna(0).astype(int)
filtered_data_cnn["cnn_input"] = filtered_data_cnn["cnn_input"].where(filtered_data_cnn["cnn_input"] <= 10000, 10000)
cnn_2nd_input = np.sum(filtered_data_cnn["cnn_input"])
cnn_2nd_input_cost = cnn_2nd_input/1000000*price_1M_input
cnn_2nd_output_cost = len(filtered_data_cnn)/1000000*price_1M_output*2 #output is just ~2 tokens
print(f"Total input cost for determening wether the {len(filtered_data_cnn):,} filtered CNN articles are positive or negative: {cnn_2nd_input_cost:.2f} USD")
print(f"Total output cost for determening wether the {len(filtered_data_cnn):,} filtered CNN articles are positive or negative: {cnn_2nd_output_cost:.4f} USD")
print("="*120)
filtered_data_fox = data_fox.loc[data_fox["is_economy"] == 1]
filtered_data_fox["fox_input"] = filtered_data_fox["body"].str.len().fillna(0).astype(int)
filtered_data_fox["fox_input"] = filtered_data_fox["fox_input"].where(filtered_data_fox["fox_input"] <= 10000, 10000)
fox_2nd_input = np.sum(filtered_data_fox["fox_input"])
fox_2nd_input_cost = fox_2nd_input/1000000*price_1M_input
fox_2nd_output_cost = len(filtered_data_fox)/1000000*price_1M_output*2 #output is just ~2 tokens
print(f"Total input cost for determening wether the {len(filtered_data_fox):,} filtered CNN articles are positive or negative: {fox_2nd_input_cost:.2f} USD")
print(f"Total output cost for determening wether the {len(filtered_data_fox):,} filtered CNN articles are positive or negative: {fox_2nd_output_cost:.4f} USD")

Total input cost for determening wether the 2,322 filtered CNN articles are positive or negative: 0.45 USD
Total output cost for determening wether the 2,322 filtered CNN articles are positive or negative: 0.0007 USD
Total input cost for determening wether the 638 filtered CNN articles are positive or negative: 0.13 USD
Total output cost for determening wether the 638 filtered CNN articles are positive or negative: 0.0002 USD


## II. S3

First 50 TB / Month	$0.023 per GB

Temp files can be deleted after running.

|File|Size|
|---|---|
cnn_economic_sentiment.jsonl | 13.9 MB
fox_news_economic_sentiment.jsonl | 6.3 MB
monthly_topic_breakdown.csv | 0.004 MB
monthly_combined_sentiments.csv | 0.0012 MB
**Total** | **20.2052**

= 0.0202052 GB

Total price/month 0.0202052*0.023 =~0.0004647 USD/month

## III. Athena

SQL queries	$5.00 per TB of data scanned.

|File|Size|
|---|---|
cnn_economic_sentiment.jsonl | 13.9 MB
fox_news_economic_sentiment.jsonl | 6.3 MB
**Total** | **20.2 MB**

Total price/refresh = 0.0000202*5 = 0.000101 USD



## IV. Lambda

Memory | Price
-|-
128MB | $0.0000000021

Billed Duration: 6245 ms	Memory Size: 128 MB	Max Memory

Total price/refresh: 6245*0.0000000021 =  0.0000131145 USD



In [58]:
6245*0.0000000021

1.3114500000000001e-05

## V. Total price

Running the whole script once (assuming it was not run before), it would cost:

In [67]:
print(f"Running the whole script once (assuming it was not run before), it would cost: ~{0.51+0.0004+0.24+0.0002+0.45+0.0007+0.13+0.002+0.0004647+0.000101:.3f} USD")

Running the whole script once (assuming it was not run before), it would cost: ~1.334 USD
